In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import sklearn


from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.cross_validation import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

import utils_veolia


/Users/christophelanternier/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
input_raw, output_raw, input_submission = utils_veolia.load_data()

In [3]:
input_preprocessed = utils_veolia.preprocess(input_raw, year=2014, more_features=False)

In [4]:
input_train, output_train, input_test, output_test = utils_veolia.split_train_test_Kfold(output_raw, input_preprocessed)

In [5]:
input_train_duplicate, output_train_duplicate = utils_veolia.data_augmentation_basic(input_train, output_train, repetitions = 6)

# Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
RFC = RandomForestClassifier(n_estimators=10, max_depth=100)
GBC = GradientBoostingClassifier(n_estimators=10, max_depth=20)

In [33]:
YEAR = 2014
input_preprocessed = utils_veolia.preprocess(input_raw, year=YEAR, more_features=False)
input_train, output_train, input_test, output_test = utils_veolia.split_train_test_Kfold(output_raw, input_preprocessed)
input_train_duplicate, output_train_duplicate = utils_veolia.data_augmentation_basic(input_train, output_train, repetitions = 6)

X_train = input_train_duplicate
utils_veolia
Y_train = utils_veolia.preprocess_output(output_train_duplicate, year = YEAR)
X_test = input_test
Y_test_2014 = utils_veolia.preprocess_output(output_test, year = YEAR)

rdm_1 = GBC
rdm_1.fit(X_train, Y_train)

y_pred_2014 = rdm_1.predict_proba(X_test)

#print(classification_report(Y_test_2014,y_pred_2014))

In [32]:
YEAR = 2015
input_preprocessed = utils_veolia.preprocess(input_raw, year=YEAR, more_features=False)
input_train, output_train, input_test, output_test = utils_veolia.split_train_test_Kfold(output_raw, input_preprocessed)
input_train_duplicate, output_train_duplicate = utils_veolia.data_augmentation_basic(input_train, output_train, repetitions = 6)

X_train = input_train_duplicate
Y_train = utils_veolia.preprocess_output(output_train_duplicate, year = YEAR)
X_test = input_test
Y_test_2015 = utils_veolia.preprocess_output(output_test, year = YEAR)

rdm_1 = GBC
rdm_1.fit(X_train, Y_train)

y_pred_2015 = rdm_1.predict_proba(X_test)

#print(classification_report(Y_test_2015,y_pred_2015))

In [37]:
y_pred_2014

array([[ 0.92595996,  0.07404004],
       [ 0.92595996,  0.07404004],
       [ 0.92595996,  0.07404004],
       ..., 
       [ 0.92595996,  0.07404004],
       [ 0.92595996,  0.07404004],
       [ 0.92595996,  0.07404004]])

In [35]:
pred = np.array([y_pred_2014,y_pred_2015]).T
true = np.array([Y_test_2014,Y_test_2015]).T

utils_veolia.score_function(true, pred)

ValueError: Found input variables with inconsistent numbers of samples: [9713, 2]

In [29]:
pred

array([[0, 0],
       [0, 0],
       [0, 0],
       ..., 
       [0, 0],
       [0, 0],
       [0, 0]])

In [30]:
true

array([[0, 0],
       [0, 0],
       [0, 0],
       ..., 
       [0, 0],
       [0, 0],
       [0, 0]])